# start2impact - progetto Python e NumPy

Ciao! Sono Emanuele Immesi e questa è la mia versione del progetto "fileorganizer" per il corso Python e NumPy del coach Francesco Bagattini.

Il progetto è suddiviso in tre step. 

## Importazioni
importo tutti i moduli necessari a svolgere il progetto.

In [1]:
import os
import shutil
import csv
from PIL import Image
import numpy as np
from tabulate import tabulate

## Step 1

### Impostazione preliminare

Uso il metodo *os.getcwd()* per ottenere l'absolute file path dove è presente il notebook e l'assegno alla costante **CWD**. 

Dichiaro una costante **DESIRED_PATH** dove è specificato il percorso della cartella 'files' che contiene i file su cui mi interessa lavorare. 

In [2]:
CWD = os.getcwd()
DESIRED_PATH = CWD + '/files'
os.chdir(DESIRED_PATH)

# Se dalla stampa sottostante si ottiene un messaggio di errore, questo potrebbe essere dovuto
# al fatto che la cartella 'files' non esiste. Procedere quindi creandola e inserire al suo interno
# tutti i files su cui lavorare.
print(os.getcwd())


C:\Users\emanu\anaconda3\envs\Lele\FileOrganizer_Progetto_Python_e_Numpy_s2i\FileOrganizer\files


### Scrivo uno script per creare le opportune sottocartelle *audio*, *images*, *docs* dove spostarvi i file appropriati ed aggiornare il documento *recap.csv*.


- Alla variabile **files_list** è associato il metodo *os.listdir()*, per generare una lista di tutti i file presenti nella cartella di lavoro
- Il dizionario **extensions** presenta chiavi con nomi uguali alle sottocartelle in cui spostare i file, mentre i valori specificati dalle chiavi corrispondono alle estensioni dei file su cui sto lavorando.
- Il primo *for loop* mi permette di creare le sottocartelle **files_destination_folder**, con nomi uguali a quello delle chiavi del dizionario, dopo aver verificato la loro esistenza con il metodo *os.path.isdir()*
- Con lo statement *with* apro un file denominato **recap.csv**, impostando la sua modalità di funzionamento su *'a' (append)* poiché voglio che lo script aggiunga voci nel file di recap per ogni utilizzo, senza formattare l'eventuale elenco preesistente. 
    - All'interno di questo statement si svolge il 'core' della funzione richiesta nello step 1. 

Vedi i commenti dentro la cella sottostante per maggiori dettagli

In [3]:
files_list = sorted(os.listdir())

extensions = {
    "image": [".jpg", ".jpeg", ".png"],
    "doc": [".doc", ".txt", ".odt"],
    "audio": [".mp3"]
    }

# Questo primo for loop è necessario a creare le sottocartelle 'image', 'doc' e 'audio' se non sono
# già presenti all'interno della cartella 'files'.
for key in extensions:
    files_destination_folder = key

    if not os.path.isdir(files_destination_folder):
        os.mkdir(files_destination_folder)    

with open('recap.csv', 'a') as csv_file:
    
    # Questi valori saranno utilizzati come 'header' del file recap.csv
    fieldnames = ['name','type','size(B)']
    
    # Per comodità ho scelto di scrivere le informazioni all'interno del file con la funzione DictWriter()
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Verifico con os.path.getsize() se il file 'recap.csv' è vuoto. In caso affermativo come prima riga viene 
    # scritto un header in cui sono presenti i campi specificati da 'fieldnames'. 
    if os.path.getsize('recap.csv') == 0:
        csv_writer.writeheader()
    
    # Il prossimo for loop itera su ogni elemento che sia esplicitamente un file presente nella lista 'files_list'. 
    for file in files_list:
                
        if os.path.isfile(file):
            
            # Grazie ai metodi os.path.splitext() e os.path.getsize() il nome, l'estensione e la dimensione (in Byte)
            # di ogni 'file' sono racchiusi rispettivamente nelle variabili 'filename', 'file_ext' e 'file_size'.
            filename, file_ext = os.path.splitext(file)
            file_size = os.path.getsize(file)
            
            # Utilizzo nuovamente questo for loop per sfruttare l'associazione tra il formato di estenzione del file sul quale
            # il ciclo sta iterando e il nome della sottocartella in cui spostare il file.
            for key in extensions:
                
                # Se l'estensione del file sul quale il ciclo sta iterando è presente nella lista di valori specificata dalla 
                # chiave attuale, allora:
                if file_ext in extensions[key]:
                    
                    # 1. Il nome della sottocartella attuale prende il nome della chiave;
                    files_destination_folder = key
                    
                    # 2. Le specifiche di: nome, tipo e dimensione del file sono scritte temporaneamente in un dizionario
                    # denominato 'file_details';
                    file_details = [{          
                    'name':filename,
                    'type': key,
                    'size(B)': file_size
                        }]
                    
                    # 3. Le informazioni attuali scritte nel dizionario 'file_details' sono stampate come riga nel file 
                    # 'recap.csv' a cui è associata la variabile 'csv_writer'; 
                    csv_writer.writerows(file_details)
                    
                    # 4. Le informazioni del file sul quale il ciclo sta iterando sono stampate nell'output di questa cella;
                    print(f"{filename} type:{key} size:{file_size}B")      
                    
                    # 5. Il file attuale viene spostato nella sottocartella di competenza con il metodo shutil.move()
                    shutil.move(file, f"{files_destination_folder}/{file}")                    


bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


## Step 2

Come richiesto dalla consegna è stato creato un file *addfile.py* eseguibile dal prompt dei comandi. La funzione di questo eseguibile è analoga a quella dello script creato nello [Step 1](#Step-1). In questo caso però viene spostato un singolo file, il cui nome (estensione compresa) deve essere specificato nel cmd. 

## Step 3


### Scrivo uno script che iteri sui file contenuti nella sottocartella *images*, estrapolando informazioni su: *larghezza* e *altezza* delle immagini, *scala di grigio* (se l'immagine è in bianco e nero), e valori *RGB* o *RGBA*.

- Alla variabile **img_list** è associato il metodo *os.listdir()*, per generare una lista di tutti i file presenti nella cartella *image*
- La lista **table** raccoglie le informazioni dei file (**name, height, width, R, G, B, Alpha**) collezionate durante l'iterazione del sottostante *for loop*.
- Le librerie PIL e NumPy sono utilizzate per estrapolare le informazioni richieste. 

### Le informazioni ottenute sono utilizzate per costruire una tabella riassuntiva con la libreria *tabulate*. 

Vedi i commenti dentro la cella sottostante per maggiori dettagli


Come fatto [sopra](#Impostazione-preliminare), uso i metodi *os.getcwd()* e *os.chdir()* per spostarmi all'interno della sottocartella  che mi interessa.

In [4]:
CWD = os.getcwd()
DESIRED_PATH = CWD + '/image'
os.chdir(DESIRED_PATH)

print(os.getcwd())

C:\Users\emanu\anaconda3\envs\Lele\FileOrganizer_Progetto_Python_e_Numpy_s2i\FileOrganizer\files\image


In [5]:
img_list = os.listdir()
print(img_list)

table = []

for file in img_list:
    
    # Ottengo il nome del file. 
    name = os.path.splitext(file)[0]
    
    # Creo un oggetto img con Image.open().
    img = Image.open(file)
    
    # Creo un array NumPy usando come argomento l'oggetto immagine appena creato.
    np_img = np.array(img)
    
    # Estrapolo le informazioni di altezza e larghezza dell'immagine. 
    height = np_img.shape[0]
    width = np_img.shape[1]
    
    # ndim mi indica il numero di dimensioni di un array:
    # 1. se un'immagine è in bianco e nero il suo numero di dimensioni sarà 2, equivalenti ad altezza e larghezza della foto; 
    if np_img.ndim < 3:
        
        # Estrapolo il valore medio della scala dei grigi.
        grayscale = np.mean(np_img)
        
        # Imposto i valori dei canali R, G, B, A uguali a 0.
        R, G, B, A = (0,0,0,0)
            
    # 2. se un'immagine è a colori avrà 3 dimensioni e la terza dimensione presenterà solo 3 elementi (RGB) sarà 
    # un'immagine RGB; 
    elif np_img.ndim == 3 and np_img.shape[2] == 3:
        
        # In questo caso quindi la scala dei grigi sarà impostata a 0.
        grayscale = 0
        
        # Collassando i valori su un unica riga e colonna e specificando quale elemento della tupla voglio ottenere, 
        # posso ricavare i valori R, G, B. 
        R, G, B = np.mean(np_img, axis=(0,1))
        
        # Il canale Alpha è assente e viene impostato a 0.
        A = 0
        
    # 3. Infine, se un immagine a colori presenta 4 canali nella terza dimensione significa che è presente anche il canale
    # Alpha, trattandosi di un'immagine RGBA. 
    else:
        
        # Come per le immagini RGB anche in questo caso la scala dei grigi viene impostata a 0.
        grayscale = 0
        
        # Come prima estrapolo le informazioni per R, G, B e in questo caso anche per il canale Alpha. 
        R, G, B, A = np.mean(np_img, axis=(0,1))
    
    # Inserisco le informazioni ottenute nella lista table. 
    table.append([name, height, width, grayscale, R, G, B, A])    

# Uso la funzione tabulate per stampare una tabella delle informazioni collezionate nella lista table. 
# Con 'headers=' imposto i titoli delle colonne dei valori, con 'tablefmt=' il formato desiderato e con 'floatfmt=' il numero di
# cifre decimali desiderate. 
print(tabulate(table, headers=["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"], tablefmt="grid", floatfmt=".2f"))


['bw.png', 'daffodil.jpg', 'eclipse.png', 'trump.jpeg']
+----------+----------+---------+-------------+--------+--------+-------+---------+
| name     |   height |   width |   grayscale |      R |      G |     B |   ALPHA |
+==========+==========+=========+=============+========+========+=======+=========+
| bw       |      512 |     512 |       21.48 |   0.00 |   0.00 |  0.00 |    0.00 |
+----------+----------+---------+-------------+--------+--------+-------+---------+
| daffodil |      500 |     335 |        0.00 | 109.25 |  85.56 |  4.97 |    0.00 |
+----------+----------+---------+-------------+--------+--------+-------+---------+
| eclipse  |      256 |     256 |        0.00 | 109.05 | 109.52 | 39.85 |  133.59 |
+----------+----------+---------+-------------+--------+--------+-------+---------+
| trump    |      183 |     275 |        0.00 |  97.01 |  98.99 | 90.92 |    0.00 |
+----------+----------+---------+-------------+--------+--------+-------+---------+


## Link utili

- start2impact sito web [qui](https://www.start2impact.it/?utm_source=google&utm_medium=cpc&utm_campaign=Search_Brand&gclid=CjwKCAjwu_mSBhAYEiwA5BBmf3Np2Y0YDvENk7J6TNUhqtt_a8-pw0qbI2Ie6RLAXavMdBo7kZQfFxoCLOUQAvD_BwE).

- Profilo di Francesco Bagattini su LinkedIn [qui](https://www.linkedin.com/in/francescobagattini/).

I miei profili:
- Profilo GitHub [qui](https://github.com/TheHextech);
- Profilo LinkedIn [qui](https://www.linkedin.com/in/emanuele-immesi-5004141b9/);
- Profilo Twitter [qui](https://twitter.com/EmanueleImmesi).